In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
import kagglehub
path = kagglehub.dataset_download("shayanfazeli/heartbeat")
print("Path to dataset files:", path)

Using Colab cache for faster access to the 'heartbeat' dataset.
Path to dataset files: /kaggle/input/heartbeat


In [ ]:
class ECGDataset(Dataset):
    def __init__(self, csv_path, mean=None, std=None, fit_stats=False):
        df = pd.read_csv(csv_path, header=None)
        data = df.values
        X = data[:, :-1].astype(np.float32)   # 187 features
        y = data[:, -1].astype(np.int64)      # labels 0..4

        if fit_stats:
            # compute normalization stats from training data only
            self.mean = X.mean(axis=0, keepdims=True)
            self.std = X.std(axis=0, keepdims=True) + 1e-8
        else:
            self.mean = mean
            self.std = std

        X = (X - self.mean) / self.std

        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [ ]:
train_csv = os.path.join(path, "mitbih_train.csv")
test_csv  = os.path.join(path, "mitbih_test.csv")

tmp_train = pd.read_csv(train_csv, header=None).values
train_mean = tmp_train[:, :-1].astype(np.float32).mean(axis=0, keepdims=True)
train_std = tmp_train[:, :-1].astype(np.float32).std(axis=0, keepdims=True) + 1e-8

train_dataset = ECGDataset(train_csv, mean=train_mean, std=train_std, fit_stats=False)
test_dataset  = ECGDataset(test_csv,  mean=train_mean, std=train_std, fit_stats=False)

batch_size = 256

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, drop_last=False)

num_features = train_dataset.X.shape[1]   # should be 187
num_classes = len(torch.unique(train_dataset.y))  # should be 5


In [ ]:
#Addressing class imbalance using weights. i.e assigning highers weights to minority classes, and vice versa
labels_np = train_dataset.y.numpy()
class_counts = np.bincount(labels_np)
class_weights = 1.0 / (class_counts + 1e-8)
class_weights = class_weights * (len(class_counts) / class_weights.sum())  # normalize a bit
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)
print("Class counts:", class_counts)
print("Class weights:", class_weights)


Class counts: [72471  2223  5788   641  6431]
Class weights: [0.02933416 0.95630948 0.36729025 3.31649917 0.33056694]


In [ ]:
#FNN Model
class ECGFNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ECGFNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.net(x)

model = ECGFNN(num_features, num_classes).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)  # you can remove "weight=..." at first
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
# ================== Training & Evaluation Loops ==================
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for X_batch, y_batch in loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * X_batch.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

    avg_loss = running_loss / total
    acc = correct / total
    return avg_loss, acc

@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_preds = []

    for X_batch, y_batch in loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        running_loss += loss.item() * X_batch.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

        all_labels.append(y_batch.cpu().numpy())
        all_preds.append(predicted.cpu().numpy())

    avg_loss = running_loss / total
    acc = correct / total
    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)
    return avg_loss, acc, all_labels, all_preds


In [ ]:
num_epochs = 20

for epoch in range(1, num_epochs + 1):
    # ----- FNN -----
    train_loss_fnn, train_acc_fnn = train_one_epoch(
        model, train_loader, criterion, optimizer, device
    )
    val_loss_fnn, val_acc_fnn, _, _ = evaluate(
        model, test_loader, criterion, device
    )

    print(f"Epoch {epoch:02d}")
    print(f"  [FNN] Train Loss: {train_loss_fnn:.4f}, Train Acc: {train_acc_fnn:.4f} | "
          f"Test Loss: {val_loss_fnn:.4f}, Test Acc: {val_acc_fnn:.4f}")

Epoch 01
  [FNN] Train Loss: 0.7134, Train Acc: 0.6276 | Test Loss: 0.5716, Test Acc: 0.8155
Epoch 02
  [FNN] Train Loss: 0.4893, Train Acc: 0.7776 | Test Loss: 0.5020, Test Acc: 0.8480
Epoch 03
  [FNN] Train Loss: 0.4270, Train Acc: 0.8072 | Test Loss: 0.3411, Test Acc: 0.8953
Epoch 04
  [FNN] Train Loss: 0.3956, Train Acc: 0.8324 | Test Loss: 0.3377, Test Acc: 0.8973
Epoch 05
  [FNN] Train Loss: 0.3702, Train Acc: 0.8372 | Test Loss: 0.4859, Test Acc: 0.8325
Epoch 06
  [FNN] Train Loss: 0.3416, Train Acc: 0.8458 | Test Loss: 0.3239, Test Acc: 0.9000
Epoch 07
  [FNN] Train Loss: 0.3294, Train Acc: 0.8538 | Test Loss: 0.3739, Test Acc: 0.8716
Epoch 08
  [FNN] Train Loss: 0.3204, Train Acc: 0.8602 | Test Loss: 0.3393, Test Acc: 0.8864
Epoch 09
  [FNN] Train Loss: 0.3008, Train Acc: 0.8639 | Test Loss: 0.3732, Test Acc: 0.8791
Epoch 10
  [FNN] Train Loss: 0.3027, Train Acc: 0.8657 | Test Loss: 0.3427, Test Acc: 0.8965
Epoch 11
  [FNN] Train Loss: 0.2908, Train Acc: 0.8713 | Test Loss: 0.

In [ ]:
#CNN Model
class ECGCNN1D(nn.Module):
    def __init__(self, num_classes, input_length=187):
        super(ECGCNN1D, self).__init__()

        self.features = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=5, padding=2),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),   # 187 -> 93

            nn.Conv1d(32, 64, kernel_size=5, padding=2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),   # 93 -> 46

            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),                     # length = 46
        )

        # 🔥 MPS-safe global pooling
        self.global_pool = nn.AdaptiveAvgPool1d(23)    # always works (46 → 1)

        # 128 channels * 1 time step = 128 features
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2944, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch, 187) → (batch, 1, 187)
        x = self.features(x)
        x = self.global_pool(x)  # (batch, 128, 1)
        return self.classifier(x)  # (batch, num_classes)

model2 = ECGCNN1D(num_classes=num_classes).to(device)

criterion2 = nn.CrossEntropyLoss(weight=class_weights_tensor)

optimizer2 = torch.optim.Adam(model2.parameters(), lr=1e-3)

In [ ]:
num_epochs = 20

for epoch in range(1, num_epochs + 1):
    # ----- CNN -----
    train_loss_cnn, train_acc_cnn = train_one_epoch(
        model2, train_loader, criterion2, optimizer2, device
    )
    val_loss_cnn, val_acc_cnn, _, _ = evaluate(
        model2, test_loader, criterion2, device
    )

    print(f"Epoch {epoch:02d}")
    print(f"  [CNN] Train Loss: {train_loss_cnn:.4f}, Train Acc: {train_acc_cnn:.4f} | "
          f"Test Loss: {val_loss_cnn:.4f}, Test Acc: {val_acc_cnn:.4f}")

Epoch 01
  [CNN] Train Loss: 0.4984, Train Acc: 0.7824 | Test Loss: 0.5525, Test Acc: 0.8270
Epoch 02
  [CNN] Train Loss: 0.3202, Train Acc: 0.8619 | Test Loss: 0.2211, Test Acc: 0.9374
Epoch 03
  [CNN] Train Loss: 0.2804, Train Acc: 0.8832 | Test Loss: 0.2457, Test Acc: 0.9216
Epoch 04
  [CNN] Train Loss: 0.2315, Train Acc: 0.8994 | Test Loss: 0.2820, Test Acc: 0.9051
Epoch 05
  [CNN] Train Loss: 0.2061, Train Acc: 0.9102 | Test Loss: 0.1375, Test Acc: 0.9587
Epoch 06
  [CNN] Train Loss: 0.1966, Train Acc: 0.9165 | Test Loss: 0.2215, Test Acc: 0.9240
Epoch 07
  [CNN] Train Loss: 0.1774, Train Acc: 0.9194 | Test Loss: 0.1921, Test Acc: 0.9356
Epoch 08
  [CNN] Train Loss: 0.1709, Train Acc: 0.9256 | Test Loss: 0.1527, Test Acc: 0.9530
Epoch 09
  [CNN] Train Loss: 0.1478, Train Acc: 0.9298 | Test Loss: 0.1480, Test Acc: 0.9514
Epoch 10
  [CNN] Train Loss: 0.1368, Train Acc: 0.9347 | Test Loss: 0.1776, Test Acc: 0.9437
Epoch 11
  [CNN] Train Loss: 0.1290, Train Acc: 0.9378 | Test Loss: 0.

In [ ]:
# ================== Final Evaluation: FNN ==================
fnn_test_loss, fnn_test_acc, fnn_y_true, fnn_y_pred = evaluate(
    model, test_loader, criterion, device
)

print("\n===== FNN Results =====")
print("Final Test Loss (FNN):", fnn_test_loss)
print("Final Test Accuracy (FNN):", fnn_test_acc)

print("\n[FNN] Classification Report:")
print(classification_report(fnn_y_true, fnn_y_pred, digits=4))

print("[FNN] Confusion Matrix:")
print(confusion_matrix(fnn_y_true, fnn_y_pred))





===== FNN Results =====
Final Test Loss (FNN): 0.2882586213830357
Final Test Accuracy (FNN): 0.9005116024118399

[FNN] Classification Report:
              precision    recall  f1-score   support

           0     0.9935    0.8922    0.9401     18118
           1     0.2729    0.8453    0.4126       556
           2     0.8220    0.9344    0.8746      1448
           3     0.2668    0.9074    0.4123       162
           4     0.9272    0.9826    0.9541      1608

    accuracy                         0.9005     21892
   macro avg     0.6565    0.9124    0.7188     21892
weighted avg     0.9536    0.9005    0.9195     21892

[FNN] Confusion Matrix:
[[16164  1225   262   352   115]
 [   65   470    15     2     4]
 [   29    13  1353    48     5]
 [    5     3     7   147     0]
 [    6    11     9     2  1580]]


In [ ]:
# ================== Final Evaluation: CNN ==================
cnn_test_loss, cnn_test_acc, cnn_y_true, cnn_y_pred = evaluate(
    model2, test_loader, criterion2, device
)

print("\n===== CNN Results =====")
print("Final Test Loss (CNN):", cnn_test_loss)
print("Final Test Accuracy (CNN):", cnn_test_acc)

print("\n[CNN] Classification Report:")
print(classification_report(cnn_y_true, cnn_y_pred, digits=4))

print("[CNN] Confusion Matrix:")
print(confusion_matrix(cnn_y_true, cnn_y_pred))


===== CNN Results =====
Final Test Loss (CNN): 0.11860908842214213
Final Test Accuracy (CNN): 0.9624977160606615

[CNN] Classification Report:
              precision    recall  f1-score   support

           0     0.9936    0.9662    0.9797     18118
           1     0.6789    0.8291    0.7466       556
           2     0.9102    0.9378    0.9238      1448
           3     0.3202    0.9506    0.4790       162
           4     0.9815    0.9900    0.9858      1608

    accuracy                         0.9625     21892
   macro avg     0.7769    0.9348    0.8230     21892
weighted avg     0.9743    0.9625    0.9669     21892

[CNN] Confusion Matrix:
[[17506   210   116   262    24]
 [   78   461    12     4     1]
 [   18     7  1358    60     5]
 [    5     1     2   154     0]
 [   11     0     4     1  1592]]
